In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quantium-data-analytics-dataset/QVI_purchase_behaviour.csv
/kaggle/input/quantium-data-analytics-dataset/QVI_transaction_data.xlsx


In [2]:
# Importing required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Loading Data
transactionData = pd.read_excel("/kaggle/input/quantium-data-analytics-dataset/QVI_transaction_data.xlsx")
customerData = pd.read_csv("/kaggle/input/quantium-data-analytics-dataset/QVI_purchase_behaviour.csv")

In [4]:
# Checking the shape of transacton data
transactionData.shape

(264836, 8)

We can see that there are more than `260K` data points in the transaction dataset.

In [5]:
# Checking first 5 rows of transaction data
transactionData.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


In [6]:
# Checking data type of each column of transaction data
print(transactionData.dtypes)

DATE                int64
STORE_NBR           int64
LYLTY_CARD_NBR      int64
TXN_ID              int64
PROD_NBR            int64
PROD_NAME          object
PROD_QTY            int64
TOT_SALES         float64
dtype: object


From the above column data type of transaction data, we can see that the `DATE` column is in `integer` format. First, we should change the format of `DATE` column to `Date`.

In [7]:
# Converting the data type of `DATE` column from integer to datetime
transactionData["DATE"] = pd.to_datetime(transactionData["DATE"], unit="D", origin=pd.Timestamp('1899-12-30'))

In [8]:
# Checking data type of each column of transaction data after coverted `DATE` column from `integer` to `date` format
print(transactionData.dtypes)

DATE              datetime64[ns]
STORE_NBR                  int64
LYLTY_CARD_NBR             int64
TXN_ID                     int64
PROD_NBR                   int64
PROD_NAME                 object
PROD_QTY                   int64
TOT_SALES                float64
dtype: object


Now we can see that the `DATE` column is in `datetime` format.

In [9]:
# Checking first 5 rows of transaction data after coverted `DATE` column from `integer` to `date` format
transactionData.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,2018-08-18,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


In [10]:
# Summary of product name column
transactionData["PROD_NAME"].describe()

count                                     264836
unique                                       114
top       Kettle Mozzarella   Basil & Pesto 175g
freq                                        3304
Name: PROD_NAME, dtype: object

From the above summary, we can conclude that there are 114 unique products, and ` Kettle Mozzarella   Basil & Pesto 175g` has the highest frequency, which is 3304.

In [11]:
# Extracting unique product name
unique_prod_names = transactionData['PROD_NAME'].unique()

# Splitting words using space as a delimiter and creating a DataFrame
product_words_df = pd.DataFrame({'words': pd.Series(' '.join(unique_prod_names).split(' '))})

# Displaying the DataFrame
print(product_words_df)

       words
0    Natural
1       Chip
2           
3           
4           
..       ...
818  Doritos
819    Salsa
820     Mild
821         
822     300g

[823 rows x 1 columns]


In [12]:
# Removing rows that contain digits
product_words_df = product_words_df[~product_words_df["words"].str.contains('\d')]

# Removing rows that contain special characters
product_words_df = product_words_df[~product_words_df["words"].str.contains('[^a-zA-Z]')]

# Removing rows that contain whitespace
product_words_df = product_words_df[~product_words_df["words"].str.contains('\s')]

# Removing rows that contain empty strings
product_words_df = product_words_df[product_words_df["words"] != ""]

# Resetting index
product_words_df = product_words_df.reset_index(drop=True)

product_words_df

,words
0,Natural
1,Chip
2,Compny
3,CCs
4,Nacho
...,...
432,Cut
433,Bolognese
434,Doritos
435,Salsa


In [13]:
# Counting number of times a word occur in Product name
word_frequency = product_words_df.groupby("words")["words"].count().reset_index(name="Frequency").sort_values(["Frequency"], ascending=False).reset_index(drop=True)
print(word_frequency)

       words  Frequency
0      Chips         21
1     Smiths         16
2    Crinkle         14
3        Cut         14
4     Kettle         13
..       ...        ...
163  GrnWves          1
164  Gcamole          1
165   Garden          1
166    Fries          1
167    saltd          1

[168 rows x 2 columns]


We can see that the word `Chips` occurs the most times. We can also see that the product name contains `Salsa` product, but we are only interested in the `Chips` category, so let's remove the row that contains `Salsa` product from the transaction dataset.

In [14]:
# Removing row which contain `Salsa` category
transactionData["Salsa"] = transactionData["PROD_NAME"].str.lower().str.contains("salsa")
transactionData = transactionData[transactionData["Salsa"] == False].drop(columns=["Salsa"])
transactionData = transactionData.reset_index(drop=True)
transactionData

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,2018-08-18,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
...,...,...,...,...,...,...,...,...
246737,2019-03-09,272,272319,270088,89,Kettle Sweet Chilli And Sour Cream 175g,2,10.8
246738,2018-08-13,272,272358,270154,74,Tostitos Splash Of Lime 175g,1,4.4
246739,2018-11-06,272,272379,270187,51,Doritos Mexicana 170g,2,8.8
246740,2018-12-27,272,272379,270188,42,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8


In [15]:
# Checking of numerical column
transactionData.describe()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
count,246742,246742.000000,2.467420e+05,2.467420e+05,246742.000000,246742.000000,246742.000000
mean,2018-12-30 01:19:01.211467520,135.051098,1.355310e+05,1.351311e+05,56.351789,1.908062,7.321322
min,2018-07-01 00:00:00,1.000000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.700000
25%,2018-09-30 00:00:00,70.000000,7.001500e+04,6.756925e+04,26.000000,2.000000,5.800000
50%,2018-12-30 00:00:00,130.000000,1.303670e+05,1.351830e+05,53.000000,2.000000,7.400000
75%,2019-03-31 00:00:00,203.000000,2.030840e+05,2.026538e+05,87.000000,2.000000,8.800000
max,2019-06-30 00:00:00,272.000000,2.373711e+06,2.415841e+06,114.000000,200.000000,650.000000
std,NaN,76.787096,8.071528e+04,7.814772e+04,33.695428,0.659831,3.077828


From the above summary, we can conclude that the `PROD_QTY` column contain some outliers.

In [16]:
# Checking null value in each columns
transactionData.isnull().sum()

DATE              0
STORE_NBR         0
LYLTY_CARD_NBR    0
TXN_ID            0
PROD_NBR          0
PROD_NAME         0
PROD_QTY          0
TOT_SALES         0
dtype: int64

No column contains a null value.

In [17]:
# Checking number of transaction of each quantity
transactionData["PROD_QTY"].value_counts()

PROD_QTY
2      220070
1       25476
5         415
3         408
4         371
200         2
Name: count, dtype: int64

There are two transactions where 200 packets of chips are bought in one transaction, and in the rest of the transactions, 5 or fewer packets of chips are bought. So, we should remove those transactions where 200 packets of chips are bought.